Importamos las librerías Pandas y Numpy

In [1]:
import pandas as pd
import numpy as np

### Step 2. Tomaremos el fichero en [esta URL](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv) y lo leeremos como un dataframe.

Pista: [read_csv](https://numpy.org/doc/stable/user/absolute_beginners.html#importing-and-exporting-a-csv)

In [2]:
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
pedidos = pd.read_csv(url, sep="\t")
pedidos.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


### Step 3. Veamos los tipos de datos. ¿Podríamos obtener el producto de mayor precio?

In [3]:
# usando el map
pedidos["item_price"].map(lambda x:float(x.replace("$","")))

#replace
pedidos["item_price_float"] = pedidos["item_price"].str.replace("$","").astype(float)


In [4]:
# Creo una columna nueva "order_items_price" con el importe total del producto por pedido (quantity * item_float_price)
pedidos["order_items_price"] = pedidos["item_price_float"] * pedidos["quantity"]
pedidos.head()

,order_id,quantity,item_name,choice_description,item_price,item_price_float,order_items_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,2.39,2.39
1,1,1,Izze,[Clementine],$3.39,3.39,3.39
2,1,1,Nantucket Nectar,[Apple],$3.39,3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98,33.96


In [14]:
# obtengo el producto con el precio más alto ordenandolo descendentemente por el item_price_float y cogiendo el primer elemento
pedidos.sort_values(by="item_price_float", ascending=False)[["item_name","item_price"]][0:1]

,item_name,item_price
3598,Chips and Fresh Tomato Salsa,$44.25


### Step 4. ¿Qué productos cuestan más de $10?

In [15]:
producto_precio_mayor_10 = pedidos[pedidos["item_price_float"]>10]
producto_precio_mayor_10.head()

,order_id,quantity,item_name,choice_description,item_price,item_price_float,order_items_price
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,16.98,33.96
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98,10.98,10.98
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75,11.75,11.75
13,7,1,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",$11.25,11.25,11.25
23,12,1,Chicken Burrito,"[[Tomatillo-Green Chili Salsa (Medium), Tomati...",$10.98,10.98,10.98


### Step 4.1: ¿Y cuántos pedidos se han hecho con un producto de más de 10$? ¿Es lo mismo?

In [16]:
#necesito los order_ids distintos a partir de los datos con el precio unitario mayor a 10$
#Los productos con un precio superior a 10$ ya los tengo filtrados en el 
# ejercicio antrior y se han guardado en producto_precio_mayor_10
numero_pedidos = len(producto_precio_mayor_10["order_id"].unique())
print("Número de pedidos con algún producto que valga más de 10$:", numero_pedidos)


Número de pedidos con algún producto que valga más de 10$: 863


### Step 4.2: ¿Y cuántos pedidos se han hecho de más de 10$? ¿Es lo mismo?

In [17]:

#presupungo que estamos hablando del importe total por pedido "order_id"
#agrupamos por pedido "order_id" y sumamos la columna "order_items_price" que tiene el total del importe por producto (quantity x unit_price).
order_price = pedidos.groupby("order_id")[["order_items_price"]].sum()
print("Precio por pedido:\n",order_price.head())
#filtramos por los que tienen una suma mayor de 10
orders_gt_10 = order_price[order_price["order_items_price"]>10]
print(orders_gt_10.head())
print("Número de pedidos de más de 10$:", len(orders_gt_10))

Precio por pedido:
           order_items_price
order_id                   
1                     11.56
2                     33.96
3                     12.67
4                     21.00
5                     13.70
          order_items_price
order_id                   
1                     11.56
2                     33.96
3                     12.67
4                     21.00
5                     13.70
Número de pedidos de más de 10$: 1834


### Step 4.3: ¿Y en cuántos pedidos se ha pagado más de 10$ por un mismo producto? ¿Es lo mismo?

In [18]:
#la columna order_items_price ya tiene el valor total pagado por el producto en un pedido (quantity x unit_price)
#voy a aplicar el filtro order_items_price
item_sum_price_gt_10 = pedidos[pedidos["order_items_price"]>10]
#ahora saco los order_ids unicos y los cuento
print(
    "Número de pedidos que se ha pagado más de 10$ por un mismo producto:",
    len(item_sum_price_gt_10["order_id"].unique())
    )

Número de pedidos que se ha pagado más de 10$ por un mismo producto: 889


### Step 5. ¿Qué precio tiene cada producto en distintos pedidos? ¿Hay productos con varios precios?

In [21]:
# obtengo un dataframe con los productos y precios posibles.
# Para ello agrupo por esas dos columnas (me da igual la operación realizada)
item_prices = pedidos.groupby(["item_name","order_items_price"]).count()
item_prices = pd.DataFrame(item_prices.index.to_list(), columns=["item_name", "order_items_price"])
item_prices.head()



,item_name,order_items_price
0,6 Pack Soft Drink,6.49
1,6 Pack Soft Drink,25.96
2,Barbacoa Bowl,8.69
3,Barbacoa Bowl,8.99
4,Barbacoa Bowl,9.25


In [20]:
#ya tengo el dataframe con productos y precios, una fila por cada producto-precio distinto.
#voy a agrupar por producto y contar el número de elementos.
item_prices_count = item_prices.groupby(["item_name"], ).count().rename(columns={"order_items_price":"prices_count"})
print("Numero de precios distintos por producto:\n", item_prices_count.head())
#los productos con más de un precio son los productos con valor mayor a 1 en la coumna "prices_count"
item_prices_count[item_prices_count["prices_count"] > 1]


Numero de precios distintos por producto:
                        prices_count
item_name                          
6 Pack Soft Drink                 2
Barbacoa Bowl                     6
Barbacoa Burrito                  6
Barbacoa Crispy Tacos             5
Barbacoa Salad Bowl               2


,prices_count
item_name,
6 Pack Soft Drink,2
Barbacoa Bowl,6
Barbacoa Burrito,6
Barbacoa Crispy Tacos,5
Barbacoa Salad Bowl,2
Barbacoa Soft Tacos,4
Bottled Water,10
Bowl,2
Canned Soda,3


### Step 6. Ordena el dataframe en base al nombre de producto (item name)

In [25]:
pedidos.sort_values("item_name").head()

,order_id,quantity,item_name,choice_description,item_price,item_price_float,unit_price
3389,1360,2,6 Pack Soft Drink,[Diet Coke],$12.98,12.98,6.49
341,148,1,6 Pack Soft Drink,[Diet Coke],$6.49,6.49,6.49
1849,749,1,6 Pack Soft Drink,[Coke],$6.49,6.49,6.49
1860,754,1,6 Pack Soft Drink,[Diet Coke],$6.49,6.49,6.49
2713,1076,1,6 Pack Soft Drink,[Coke],$6.49,6.49,6.49


### Step 7. ¿Cuantas veces se ha pedido los productos más caros?

In [22]:
# agrupo por producto y precio y cuento el número de "order_id" que ha habido.
# renombro la columna donde se guardará el dato acumulado con el nombre "num_pedidos"
numero_pedidos_por_producto_precio = pedidos.groupby(
        ["item_name",
        "order_items_price"]
    ).count().rename(columns={"order_id":"num_pedidos"})[["num_pedidos"]]

#ordeno el nuevo dataframe por el precio de forma descendente.
numero_pedidos_por_producto_precio = numero_pedidos_por_producto_precio.sort_values(
        "order_items_price",
        ascending=False
    )

numero_pedidos_por_producto_precio.head()

,,num_pedidos
item_name,order_items_price,
Chips and Fresh Tomato Salsa,663.75,1
Bottled Water,150.00,1
Chicken Burrito,140.00,2
Side of Chips,108.16,1
Carnitas Bowl,105.75,1


### Step 8. Veamos para el caso de Veggie Salad Bowl. Extrae esa información.

In [27]:
# Sacamos la información del dataframe calculado en el ejercicio anterior filtrandolo por el nombre del producto
numero_pedidos_por_producto_precio.loc["Veggie Salad Bowl"].head()

,num_pedidos
unit_price,
11.25,10
8.75,8
